In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
%matplotlib inline 

In [ ]:
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, f1_score


In [ ]:
df = pd.read_csv("/content/train2.csv", encoding='cp1251', on_bad_lines='skip', sep = ';')
df.tail()

In [ ]:
df.info()

In [ ]:
df['Возраст']=df['Возраст'].replace('unknown',np.nan)
df=df.dropna(axis=0, subset=['Возраст'])
print(df.tail())

In [ ]:
print(len(df))
print(df.shape)
a=df['Возраст'].value_counts()[:18]
b=a.keys().tolist()
print(b)
df=df[df.Возраст.isin(b)]
df=df.reset_index(drop=True)

In [ ]:
genre_count_df = df[['Название книги','Возраст']].groupby('Возраст').agg('count').sort_values('Название книги', ascending = False)

genre_count_df.reset_index(inplace = True)
genre_count_df.rename(columns ={'Название книги':'Возраст_cnt'}, inplace = True)

genre_count_df.plot.bar(y = 'Возраст_cnt', x = 'Возраст')

genre_count_df

In [ ]:
df = df.merge(genre_count_df, how = 'left', left_on='Возраст', right_on='Возраст')

df

In [ ]:
CNT_MIN = 50

df = df [df['Возраст_cnt'] > CNT_MIN]

df

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
def plotToWords(raw_plot):
    letters_only = re.sub("[^а-яА-Я]", " ", str(raw_plot))
    lower_case = letters_only.lower()
    words = lower_case.split()
    stops = set(stopwords.words("russian"))
    meaningful_words = [w for w in words if not w in stops]
    return (" ".join(meaningful_words))

def preprocess(dataframe):
    clean_train_reviews = []
    for i in range(0,len(dataframe)):
        clean_train_reviews.append(plotToWords(dataframe.iloc[i]['Аннотация']))
    dataframe['Аннотация']=clean_train_reviews
    return dataframe

df=preprocess(df)
print(df["Аннотация"][:10])

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=1, norm='l2', encoding='latin-1', ngram_range=(1, 2), max_features=4000)
features = tfidf.fit_transform(df.Аннотация).toarray()
labels = df.Возраст
features.shape

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
X_train, X_test, y_train, y_test = train_test_split(df['Аннотация'], df['Возраст'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = MultinomialNB().fit(X_train_tfidf, y_train)


In [ ]:
print(clf.predict(count_vect.transform(["Новая история происходит в вымышленном городе Южная Верона, где Монтекки и Капулетти становятся Кампелос и Монтейрос - двумя враждующими семьями, известными всему городу. Однако на этом сходство с оригинальной историей заканчивается. Ромео и Джульетта становятся Ромуло и Рамоном и впитывают черты характера обоих классических персонажей, давая новую жизнь известному сюжету."])))

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
models = [
    LinearSVC(max_iter=150000),
    MultinomialNB(),
    LogisticRegression(random_state=0,max_iter=150000),
]
CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
    model_name = model.__class__.__name__
    accuracies = cross_val_score(model, features, labels, scoring='accuracy', cv=CV)
    for fold_idx, accuracy in enumerate(accuracies):
        entries.append((model_name, fold_idx, accuracy))
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])

In [ ]:
cv_df.groupby('model_name').accuracy.mean()